https://groups.google.com/g/the-art-of-multiprocessor-programming-2nd-edition

[](/_notes/CS/Concurrency.md)
https://lobste.rs/s/va2mvi/without_futex_it_s_futile


Lamport book  https://lamport.azurewebsites.net/tla/book-21-07-04.pdf   https://lamport.azurewebsites.net/tla/high-level-view.html
https://eatonphil.com/bookclub.html

Lynch distributed algorithms

https://cs.brown.edu/courses/cs176/ 

https://harmony.cs.cornell.edu/book/ https://harmony.cs.cornell.edu/ https://github.com/harmonylang/harmony

https://github.com/fizzbee-io/fizzbee
alloy https://practicalalloy.github.io/chapters/behavioral-modeling/index.html
ivy?
quint


# chapter 1 intro
Alice and Bob cats and Dogs

Prdoucer consumer


In [ ]:
%%file /tmp/configfile.cfg
SPECIFICATION Spec

INVARIANT Inv1, Inv2
PROPERTY Prop1

In [ ]:
%%file /tmp/alias.tla
---- MODULE aliases ----
EXTENDS Integers

VARIABLE x
Init ==
  x = 0

Next == x' = x + 1
Inv == x < 10
Spec == Init /\ [][Next]_x

Alias ==
  [x |-> x,
   nextx |-> x',
   incx |-> x + 1]
=====

In [15]:
%%file /tmp/hello.java

void main() {
    System.out.println("Hello, World!");
}

Writing /tmp/hello.java


Amdahl's law

fraction that can be aprallized p
1 - p + p/n

In [22]:
! java --enable-preview --source 21 /tmp/hello.java

Note: /tmp/hello.java uses preview features of Java SE 21.
Note: Recompile with -Xlint:preview for details.
Hello, World!


https://docs.oracle.com/javase/8/docs/api/java/util/concurrent/package-summary.html

In [ ]:
%%file /tmp/primePrint.java

   void main() {
      int i = ThreadID.get(); // thread IDs are in {0..9}
      long block = power(10, 9);
      for ( long j = (i * block) + 1; j <= (i + 1) * block; j++) {
   if (isPrime(j))
      print(j);
   }
   }



Overwriting /tmp/primePrint.java


In [23]:
! java --enable-preview --source 21 /tmp/primePrint.java 

/tmp/primePrint.java:4: error: cannot find symbol
      int i = ThreadID.get(); // thread IDs are in {0..9}
              ^
  symbol:   variable ThreadID
  location: class primePrint
/tmp/primePrint.java:5: error: cannot find symbol
      long block = power(10, 9);
                   ^
  symbol:   method power(int,int)
  location: class primePrint
/tmp/primePrint.java:7: error: cannot find symbol
   if (isPrime(j))
       ^
  symbol:   method isPrime(long)
  location: class primePrint
/tmp/primePrint.java:8: error: cannot find symbol
      print(j);
      ^
  symbol:   method print(long)
  location: class primePrint
4 errors
error: compilation failed


# chapter 2 Mutual Exclusion
Bakery algorithm
Peterson Lock

https://en.wikipedia.org/wiki/Peterson%27s_algorithm

https://jamesrwilcox.com/SharedMem.html proof of peterson in coq
https://homes.cs.washington.edu/~mernst/pubs/verify-distsystem-pldi2015.pdf verdi

https://harmony.cs.cornell.edu/book/#sec-33

https://cs.brown.edu/~tbn/publications/peterson.pdf alloy peterson

https://forge-fm.org/

ivy?
Transitive clousre vs int parametrized.

https://stackoverflow.com/questions/11588514/a-tested-implementation-of-peterson-lock-algorithm
https://bartoszmilewski.com/2008/11/05/who-ordered-memory-fences-on-an-x86/

https://ocw.mit.edu/courses/6-172-performance-engineering-of-software-systems-fall-2018/resources/lecture-17-synchronization-without-locks/

mutual exclusion
no deadlock - if multuple try, at least one succeeds
no starvation - if attempt, must eventually succeed

In [3]:
%%file /tmp/lock1.pml

bool flag[2];
byte ncrit;

active [2] proctype user()
{
	assert(_pid == 0 || _pid == 1);
again:
	flag[_pid] = 1;
	flag[1 - _pid] == 0;

	ncrit++;
	assert(ncrit == 1);	/* critical section */
	ncrit--;

	flag[_pid] = 0;
	goto again
}

Writing /tmp/lock1.pml


In [5]:
! spin -v /tmp/lock1.pml

  1:	proc  0 (user:1) /tmp/lock1.pml:7 (state 1)	[assert(((_pid==0)||(_pid==1)))]
  2:	proc  1 (user:1) /tmp/lock1.pml:7 (state 1)	[assert(((_pid==0)||(_pid==1)))]
  3:	proc  1 (user:1) /tmp/lock1.pml:9 (state 2)	[flag[_pid] = 1]
  4:	proc  1 (user:1) /tmp/lock1.pml:10 (state 3)	[((flag[(1-_pid)]==0))]
  5:	proc  0 (user:1) /tmp/lock1.pml:9 (state 2)	[flag[_pid] = 1]
  6:	proc  1 (user:1) /tmp/lock1.pml:12 (state 4)	[ncrit = (ncrit+1)]
  7:	proc  1 (user:1) /tmp/lock1.pml:13 (state 5)	[assert((ncrit==1))]
  8:	proc  1 (user:1) /tmp/lock1.pml:14 (state 6)	[ncrit = (ncrit-1)]
  9:	proc  1 (user:1) /tmp/lock1.pml:16 (state 7)	[flag[_pid] = 0]
 10:	proc  1 (user:1) /tmp/lock1.pml:17 (state 8)	[goto again]
 11:	proc  0 (user:1) /tmp/lock1.pml:10 (state 3)	[((flag[(1-_pid)]==0))]
 12:	proc  0 (user:1) /tmp/lock1.pml:12 (state 4)	[ncrit = (ncrit+1)]
 13:	proc  1 (user:1) /tmp/lock1.pml:9 (state 2)	[flag[_pid] = 1]
 14:	proc  0 (user:1) /tmp/lock1.pml:13 (state 5)	[assert((ncrit==1))]
 15:	pro

In [1]:
%%file /tmp/peterson.pml
// https://github.com/nimble-code/Spin/blob/master/Examples/peterson.pml
/* Peterson's solution to the mutual exclusion problem - 1981 */

bool turn, flag[2];
byte ncrit;

active [2] proctype user()
{
	assert(_pid == 0 || _pid == 1);
again:
	flag[_pid] = 1;
	turn = _pid;
	(flag[1 - _pid] == 0 || turn == 1 - _pid);

	ncrit++;
	assert(ncrit == 1);	/* critical section */
	ncrit--;

	flag[_pid] = 0;
	goto again
}

Writing /tmp/peterson.pml


In [17]:
%%file /tmp/Peterson.tla
---- MODULE Peterson ----
EXTENDS Integers
(*
--algorithm Peterson {
    variables flag = [i \in {0, 1} |-> FALSE],  turn = 0;
        \* Declares the global variables flag and turn and their initial values;
        \* flag is a 2-element array with initially flag[0] = flag[1] = FALSE.
    fair process (proc \in {0,1}) {
    \* Declares two processes with identifier self equal to 0 and 1.
    \* The keyword fair means that no process can stop forever if it can
    \* always take a step.
    a1: while (TRUE) {
            skip ;  \* the noncritical section
    a2:  flag[self] := TRUE ;
    a3:  turn := 1 - self ;   
    a4:  await (flag[1-self] = FALSE) \/ (turn = self); \* \/ is written || in C.
    cs:  skip ;  \* the critical section
    a5:  flag[self] := FALSE               }     }     }
*)
====

Overwriting /tmp/Peterson.tla


In [18]:
! java -cp ~/Downloads/tla2tools.jar pcal.trans /tmp/Peterson.tla && cat /tmp/Peterson.tla

pcal.trans Version 1.11 of 31 December 2020
Parsing completed.
Translation completed.
New file /tmp/Peterson.tla written.
New file /tmp/Peterson.cfg written.
---- MODULE Peterson ----
EXTENDS Integers
(*
--algorithm Peterson {
    variables flag = [i \in {0, 1} |-> FALSE],  turn = 0;
        \* Declares the global variables flag and turn and their initial values;
        \* flag is a 2-element array with initially flag[0] = flag[1] = FALSE.
    fair process (proc \in {0,1}) {
    \* Declares two processes with identifier self equal to 0 and 1.
    \* The keyword fair means that no process can stop forever if it can
    \* always take a step.
    a1: while (TRUE) {
            skip ;  \* the noncritical section
    a2:  flag[self] := TRUE ;
    a3:  turn := 1 - self ;   
    a4:  await (flag[1-self] = FALSE) \/ (turn = self); \* \/ is written || in C.
    cs:  skip ;  \* the critical section
    a5:  flag[self] := FALSE               }     }     }
*)
\* BEGIN TRANSLATION (chksum(pcal) =

In [19]:
! java -cp ~/Downloads/tla2tools.jar tlc2.TLC /tmp/Peterson.tla

TLC2 Version 2.19 of 08 August 2024 (rev: 5a47802)


(Use the -nowarning option to disable this warning.)
Running breadth-first search Model-Checking with fp 111 and seed -1918630056517602505 with 1 worker on 16 cores with 15736MB heap and 64MB offheap memory [pid: 3180596] (Linux 6.14.0-28-generic amd64, Ubuntu 21.0.8 x86_64, MSBDiskFPSet, DiskStateQueue).
Parsing file /tmp/Peterson.tla
Parsing file /tmp/Integers.tla
Parsing file /tmp/Naturals.tla
Semantic processing of module Naturals
Semantic processing of module Integers
Semantic processing of module Peterson
Starting... (2025-09-03 09:28:41)
Computing initial states...
Finished computing initial states: 1 distinct state generated at 2025-09-03 09:28:41.
Model checking completed. No error has been found.
  Estimates of the probability that TLC did not check all reachable states
  because two distinct states had the same fingerprint:
  calculated (optimistic):  val = 8.0E-17
77 states generated, 42 distinct states found, 0 states left on queue.
The depth of the complete state graph se

In [20]:
%%file /tmp/lock1.tla
---- MODULE lock1 ----
EXTENDS Integers
(*

--algorithm Lock1 {
    variables flag = [i \in {0, 1} |-> FALSE];
    fair process (proc \in {0,1}) {
    a1: while (TRUE) {
            skip ;  \* the noncritical section
    a2:  flag[self] := TRUE ;
    a4:  await (flag[1-self] = FALSE);
    cs:  skip ;  \* the critical section
    a5:  flag[self] := FALSE               }     }     }
*)
====



Writing /tmp/lock1.tla


In [21]:
! java -cp ~/Downloads/tla2tools.jar pcal.trans /tmp/lock1.tla && java -cp ~/Downloads/tla2tools.jar tlc2.TLC /tmp/lock1.tla

pcal.trans Version 1.11 of 31 December 2020
Parsing completed.
Translation completed.
New file /tmp/lock1.tla written.
New file /tmp/lock1.cfg written.
TLC2 Version 2.19 of 08 August 2024 (rev: 5a47802)
(Use the -nowarning option to disable this warning.)
Running breadth-first search Model-Checking with fp 77 and seed -3099753534938489767 with 1 worker on 16 cores with 15736MB heap and 64MB offheap memory [pid: 3187713] (Linux 6.14.0-28-generic amd64, Ubuntu 21.0.8 x86_64, MSBDiskFPSet, DiskStateQueue).
Parsing file /tmp/lock1.tla
Parsing file /tmp/Integers.tla
Parsing file /tmp/Naturals.tla
Semantic processing of module Naturals
Semantic processing of module Integers
Semantic processing of module lock1
Starting... (2025-09-03 09:32:10)
Computing initial states...
Finished computing initial states: 1 distinct state generated at 2025-09-03 09:32:10.
Error: Deadlock reached.
Error: The behavior up to this point is:
State 1: <Initial predicate>
/\ flag = (0 :> FALSE @@ 1 :> FALSE)
/\ pc =

python. Use yield or await?
hypotehsis? Take cue from rust yarn libs
https://hypothesis.readthedocs.io/en/latest/stateful.html
https://hypothesis.works/articles/how-not-to-die-hard-with-hypothesis/
https://hypothesis.works/articles/rule-based-stateful-testing/ 
Seems more painful than is worth.



esbmc actually supports python. Hmmm.


pcode execution with 


In [ ]:
from hypothesis.stateful import RuleBasedStateMachine, precondition, rule
class PC(Enum):
    Init = 0
    Next = 1
    Inc = 2
    CRIT = 3

class Peterson(RuleBasedStateMachine):
    pc = [PC.Init, PC.Init]
    flag = [False, False]
    turn = 0

    @precondition(lambda self: self.pc[0] == PC.Init and self.pc[1] == PC.Init)
    @rule()
    def set_flag(self):

In [ ]:
ProcID = kd.Enum("ProcID", ["P0", "P1"])
PC = kd.Enum("PC", ["Init", "Flag1", "Flag2", "Crit", "Exit"])
Spec = smt.Function("Spec", smt.ArraySort(ProcID, smt.BoolSort()), smt.ArraySort(ProcID, smt.BoolSort()))
Spec = kd.define("Spec", [flag, pc, flag1, pc1], 
    smt.Or(
        smt.And(pc[n] == PC.Init),
        smt.And(pc[n] == PC.Flag1, flag[n] == True, pc1[n] == PC.Flag2),
    )
)

smt.TransitiveClosure(Spec)

In [ ]:
%%file /tmp/peterson.sv
module peterson(
    input clk,
    input pid,
    output reg flag[2],
    output reg pc[2]
)
@always @(posedge clk) begin
    switch pc[pid]
        PC.Init: begin
            flag[pid] <= 0;
            pc[pid] <= PC.Flag1;
        end
        PC.Flag1: begin
            flag[pid] <= 1;
            pc[pid] <= PC.Flag2;
        end
        PC.Flag2: begin
            if (flag[1 - pid] == 0) begin
                pc[pid] <= PC.Crit;
            end else begin
                pc[pid] <= PC.Flag2; // wait
            end
        end
        PC.Crit: begin
            // Critical section logic here
            pc[pid] <= PC.Exit;
        end
        PC.Exit: begin
            flag[pid] <= 0;
            pc[pid] <= PC.Init;
        end

        assert property !(pc[0] == PC.Crit && pc[1] == PC.Crit);
endmodule

In [ ]:

# 
def lock1(pid, flag):
    assert pid == 0 or pid == 1
    while True:
        yield "a0"
        flag[pid] = True
        yield "a1"
        flag[1 - pid] = False
        yield "a2"

        yield "a3"
        ncrit += 1
        yield "a4"
        assert ncrit == 1  # critical section
        yield "a5"
        ncrit -= 1

        flag[pid] = False
# use hypothesis?
def search():
    while True:
        flag = [False, False]
        procs = [lock1(0, flag), lock1(1, flag)]
        trace = []
        for i in range(10): # 10 steps
            try:
                trace.append(flags.copy(), next(random.choice(procs)))
        






In [ ]:
@dataclass
class State:
    flag: Tuple[bool,bool] = (False, False)
    ncrit: int = 0

def lock(pid):
    assert pid == 0 or pid == 1
    while True:
        yield "a0"; st.flag[pid] = True
        yield "a1"; st.flag[1 - pid] = False 
        yield "a3": st.ncrit += 1
        yield "a4": assert st.ncrit == 1  # critical section
        yield "a5": st.ncrit -= 1
        yield "a6": st.flag[pid] = False

@given(st.lists(st.sampled_from([0,1]), min_size=1, max_size=60))
def test_protocol(sched):
    st = State()
    procs = [lock(pid) for pid in range(2)]
    trace = []
    for step in sched:
        trace.append((step, next(procs[step])))


In [ ]:
def t0(pid, st):
    return st
def t1(pid, st):
    return st
def t2(pid, st):

{
    "t0": lambda pid, st: st,
    "t1": lambda pid, st: st,
    "t2": lambda pid, st: st,
}

proc = [t0,t1,t2,t3]


Spacer. datalog. prolog

Why is a one sided reachablity ok now?

In [ ]:
smt.QForAll([flags, pc], 
       , step()
)

def horn(vs, head, *bodies):
    return [
        smt.ForAll(vs, smt.Implies(smt.And(body), head))
        for body in bodies
    ]
def horn([flag, pc], spec(flags, pc), 
         smt.And(pc[0] == PC.Init, spec() )
         smt.And(pc[0] == PC.Flag1, flag[0] == True, pc1[0] == PC.Flag2) 
         ,
         )

Memory models: Starting point is PC is totally random.
Starting point is everyone has their own copy of memory and PC is random? Or that everyone else's PC is random.

https://github.com/heidihoward/distributed-consensus-reading-list?tab=readme-ov-file#correctness-of-consensus-algorithms

https://github.com/PirateshipOrg/pirateship-tla wha?
